# **Setup Environment**

In [ ]:
# Change directory to the project root so that relative paths in .env 
# (e.g., model weights, data directories) resolve correctly
%cd ../

In [ ]:
import sys
from dotenv import load_dotenv
sys.path.append('.')
load_dotenv()

import json
from app.human_pose_estimator import (
    HumanPoseEstimator2D,
    HumanDetectorConfig,
    PoseEstimatorConfig
)
from app.s3 import files_service
from app.video import decode_from_minio_streaming
from app.running_analysis import analyze_full_video, AnatomicalSide
from app.pickle_data import (
    save_estimations_to_minio, 
    load_estimations_from_minio, 
    save_analysis_to_minio, 
    load_analysis_from_minio
)

human_detector_config = HumanDetectorConfig()
pose_estimator_config = PoseEstimatorConfig()

In [ ]:
# Start MinIO — required by code that interacts with object storage
!pixi run just minio-up

In [ ]:
files_service.create_bucket()

In [ ]:
# # After finishing your work, shut down MinIO to free resources
# !pixi run just minio-down

In [ ]:
# Change directory to 'assets/' so that files can be referenced directly 
# (e.g., 'test.png' instead of 'assets/test.png')
%cd playground-testing/assets/

In [ ]:
INPUT_VIDEO_PATH = "test.mp4"

VIDEO_KEY = "test4/video"
ESTIMATIONS_KEY = "test4/estimations"
FPS_TARGET = 5

ANALYSIS_KEY = "test4/analysis"
SIDE = AnatomicalSide.RIGHT
OUTPUT_ANALYSIS_PATH = "4.analysis.json"

## **Test**

In [ ]:
def test_analyze_video_and_save_results_to_minio(
    input_video_path: str,
    video_key: str,
    results_key: str,
    fps_target: float
) -> None:
    """
    Stream video from MinIO, run pose estimation and save results to MinIO as pickle.

    Parameters
    ----------
    input_video_path : str
        Local path to input video file.
    video_key : str
        MinIO key to upload the input video.
    results_key : str
        MinIO key to store the resulting pose estimation data as pickle.
    fps_target : float
        Target FPS for pose estimation.
    """

    # 1. Upload input video to MinIO
    with open(input_video_path, "rb") as f:
        files_service.upload_fileobj(storage_key=video_key, fileobj=f)

    # 2. Initialize raw frame stream from MinIO
    raw_frame_gen = decode_from_minio_streaming(
        storage_key=video_key,
        expires=300,
        fps_target=fps_target
    )

    # 3. Initialize pose estimator
    human_pose_estimator = HumanPoseEstimator2D(
        human_detector_config=human_detector_config,
        pose_estimator_config=pose_estimator_config
    )

    # 4. Run pose estimation and collect keypoints
    keypoints_sequence = []
    for frame in raw_frame_gen:
        result = human_pose_estimator(frame)
        keypoints_sequence.append(result.keypoints)

    # 5. Save results
    save_estimations_to_minio(
        results=keypoints_sequence,
        storage_key=results_key
    )

    print(f"Successfully loaded video file '{input_video_path}', estimated pose, and saved results to '{results_key}'.")

In [ ]:
test_analyze_video_and_save_results_to_minio(
    input_video_path = INPUT_VIDEO_PATH,
    video_key = VIDEO_KEY,
    results_key = ESTIMATIONS_KEY,
    fps_target = FPS_TARGET
)

In [ ]:
def test_analyze_pose_results_and_save_analysis_to_minio(
    results_key: str,
    analysis_key: str,
    side: AnatomicalSide
) -> None:
    """
    Load pose estimation results from MinIO, run biomechanical analysis, and save analysis to MinIO.

    Parameters
    ----------
    results_key : strля этих функций; проверочный код пиши через requests библиотеку ;

        MinIO key where pose estimation results (keypoints sequence) are stored.
    analysis_key : str
        MinIO key to store the resulting biomechanical analysis.
    side : AnatomicalSide
        Anatomical side to use for signed angles.
    """

    # 1. Load pose estimation results (list of KeypointMap2D)
    keypoints_sequence = load_estimations_from_minio(storage_key=results_key)

    # 2. Run full video biomechanical analysis
    analysis = analyze_full_video(keypoints_sequence, side=side)

    # 3. Save analysis results to MinIO
    save_analysis_to_minio(analysis=analysis, storage_key=analysis_key)

    print(f"Successfully loaded pose results from '{results_key}', performed biomechanical analysis, and saved to '{analysis_key}'.")

In [ ]:
test_analyze_pose_results_and_save_analysis_to_minio(
    results_key = ESTIMATIONS_KEY,
    analysis_key = ANALYSIS_KEY,
    side = SIDE
)

In [ ]:
def load_analysis_as_dict_from_minio(
    analysis_key: str,
    output_path: str
) -> dict:
    """
    Load biomechanical analysis from MinIO, convert to dictionary, and save as JSON.

    Parameters
    ----------
    analysis_key : str
        MinIO key where the biomechanical analysis (VideoAnalysis) is stored.
    output_path : str
        Local filesystem path where the resulting JSON file will be saved.

    Returns
    -------
    dict
        Dictionary representation of the VideoAnalysis NamedTuple.
    """

    # 1. Load analysis object from MinIO
    analysis = load_analysis_from_minio(storage_key=analysis_key)

    # 2. Convert to dictionary for easy serialization or inspection
    analysis_dict = analysis._asdict()

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(analysis_dict, f, indent=2, ensure_ascii=False)

    print(f"Successfully loaded biomechanical analysis from '{analysis_key}' and saved JSON to '{output_path}'.")

    return analysis_dict

In [ ]:
load_analysis_as_dict_from_minio(
    analysis_key = ANALYSIS_KEY,
    output_path = OUTPUT_ANALYSIS_PATH
)